In [1]:
import import_ipynb
from functions import get_final_answer
import requests
from bs4 import BeautifulSoup, NavigableString
import re
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_dataset
from tabulate import tabulate

importing Jupyter notebook from functions.ipynb


c:\Users\zfkha\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url = "https://en.wikipedia.org/wiki/Fox"
#wiki_extract = get_wiki_extract(url)
#print(wiki_extract)
question = "What does the fox say?"
answer = get_final_answer(question=question,url=url)
print(answer)

c:\Users\zfkha\miniconda3\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


whining


In [3]:
dataset = load_dataset("squad_v2",split="validation")

In [4]:
df = pd.DataFrame(dataset)

In [5]:
df = df.drop("id", axis=1)
df = df.drop("context",axis=1)
#df

In [ ]:
df['text'] = df['answers'].apply(lambda x: x['text'])
df = df.drop(df[df['text'].apply(lambda x: len(x) == 0)].index)
df = df.drop('answers', axis=1)
df.rename(columns = {'text':'answers'}, inplace = True)
#df

In [ ]:
df2 = df.sample(300)

In [ ]:
answers = []
for index, row in df2.iterrows():
    url = "https://en.wikipedia.org/wiki/" + row["title"]
    #print(url)
    question = row["question"]
    #print(question)
    answer = get_final_answer(question=question,url=url)
    answers.append(answer)

In [ ]:
correct_answers = pd.DataFrame(df2["answers"]).values

In [ ]:
def calculate_metrics(true_answers, predicted_answers):
    """
    Calculates the F1 score given the true answers and predicted answers.
    Both true_answers and predicted_answers are lists of strings.
    """
    # Convert predicted_answers list to a set of tuples
    predicted_answers_set = set(predicted_answers)

    # Calculate precision
    if len(predicted_answers_set) == 0:
        precision = 0
    else:
        true_positives = 0
        for ans_p in predicted_answers_set:
            for ans_t in true_answers:
                if all([a in ans_p for a in ans_t.split()]):
                    true_positives += 1
                    break  # go to next predicted answer if a true answer is found
        precision = true_positives / len(predicted_answers_set)

    # Calculate recall
    true_positives = 0
    for ans_t in true_answers:
        for ans_p in predicted_answers_set:
            if all([a in ans_t for a in ans_p]):
                true_positives += 1
                break  # go to next true answer if a predicted answer is found
    recall = true_positives / len(true_answers)

    # Calculate F1 score
    if precision == 0 and recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * precision * recall / (precision + recall)

    return precision, recall, f1_score

In [ ]:
pos = 0
neg = 0
for i in range(0,len(answers)):
    answer = answers[i]
    correct_answer = correct_answers[i]
    #print(answer)
    #print(correct_answer)
    for inner_list in correct_answer:
        for element in inner_list:
            if answer in element or element in answer:
                pos+=1
                break
            else:
                continue
            break
        else:
            neg+=1
acc_rate = 100*(pos/(pos+neg))
print("accuracy rate = ",acc_rate,"%")

In [ ]:
true_positives = 0
false_positives = 0
false_negatives = 0

for i in range(0, len(answers)):
    answer = answers[i]
    correct_answer = correct_answers[i]
    is_match = False
    for inner_list in correct_answer:
        for element in inner_list:
            if answer in element or element in answer:
                is_match = True
                break
        if is_match:
            break
    if is_match:
        true_positives += 1
    else:
        false_negatives += 1

    is_match = False
    for inner_list in correct_answer:
        for element in inner_list:
            #print(element)
            #print(str(correct_answer))
            if element in correct_answer or str(correct_answer) in element:
                is_match = True
                print(element)
                print(str(correct_answer))
                break
        if is_match:
            break
    if not is_match:
        false_positives += 1

precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)

print(true_positives)
print(false_positives)
print(false_negatives)

print("precision = ", precision)
print("recall = ", recall)
print("F1 score = ", f1_score)

In [ ]:
triviaqa = pd.read_json("training.json")

In [ ]:
triviaqa = triviaqa.drop("version",axis=1)

In [ ]:
triviaqa['title'] = triviaqa['data'].apply(lambda x: x['title'])
triviaqa = triviaqa.drop(df[df['title'].apply(lambda x: len(x) == 0)].index)
triviaqa['paragraphs'] = triviaqa['data'].apply(lambda x: x['paragraphs'][0])
triviaqa['qas'] = triviaqa['paragraphs'].apply(lambda x: x['qas'][0])
triviaqa['question'] = triviaqa['qas'].apply(lambda x: x['question'])
triviaqa['answers'] = triviaqa['qas'].apply(lambda x: x['answers'])
triviaqa = triviaqa.drop(triviaqa[triviaqa['answers'].apply(lambda x: len(x) == 0)].index)
triviaqa['answers'] = triviaqa['qas'].apply(lambda x: x['answers'][0])
triviaqa['answer'] = triviaqa['answers'].apply(lambda x: x['text'])
triviaqa = triviaqa.drop(['data','paragraphs','qas','answers'], axis=1)
#triviaqa

In [ ]:
df3 = triviaqa.sample(300)

In [ ]:
answers2 = []
questions2 = []
correct_answers2 = []
for index, row in df3.iterrows():
    #print(row["title"])
    #print(row["question"])
    url = "https://en.wikipedia.org/wiki/" + row["title"]
    #print(url)
    question = row["question"]
    #print(question)
    answer = get_final_answer(question=question,url=url)
    correct_answer = row['answer']
    correct_answers2.append(correct_answer)
    questions2.append(question)
    answers2.append(answer)

In [ ]:
print(questions2)
print(answers2)
print(correct_answers2)

In [ ]:
url = "https://en.wikipedia.org/wiki/" + "Central_Intelligence_Agency"
#print(url)
question = "What is the CIA's main focus?"
#print(question)
answer = get_final_answer(question=question,url=url)
print(answer)

In [ ]:
incorrect_questions = []
i = 0

for index, row in df3.iterrows():
    answer = row['answer']
    correct_answer = correct_answers2[i][0]

    column_value = row['question']

    if answer not in correct_answer or str(correct_answer) not in answer:
        incorrect_questions.append(column_value)
    
    i+=1

In [ ]:
table_data = list(zip(questions2, answers2, correct_answers2))

headers = ['Questions', 'Answers', 'Correct Answers']

print(tabulate(table_data, headers=headers, tablefmt='grid'))

In [ ]:
pos = 0
neg = 0
for i in range(0,df3.shape[0]):
    answer = answers2[i]
    correct_answer = correct_answers2[i][0]
    #print(answer)
    #print(correct_answer)
    if answer in correct_answer or str(correct_answer) in answer:
        pos+=1
    else:
        neg+=1
acc_rate = 100*(pos/(pos+neg))
print("accuracy rate = ",acc_rate,"%")

In [ ]:
TP = 0
FP = 0
FN = 0

for i in range(0,df3.shape[0]):
    answer = answers2[i]
    correct_answer = correct_answers2[i]
    if answer in correct_answer or str(correct_answer) in answer:
        if (answer == correct_answer).any():
            TP += 1
        else:
            FP += 1
    else:
        if answer != "":
            FN += 1

precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * precision * recall / (precision + recall)

print(TP)
print(FP)
print(FN)
print("precision = ",precision)
print("recall = ",recall)
print("f1 score = ",f1_score)

In [ ]:
true_positives = 0
false_positives = 0
false_negatives = 0

for i in range(0, len(answers2)):
    answer = answers2[i]
    correct_answer = correct_answers2[i]
    is_match = False
    for inner_list in correct_answer:
        for element in inner_list:
            if answer in element or element in answer:
                is_match = True
                break
        if is_match:
            break
    if is_match:
        true_positives += 1
    else:
        false_negatives += 1

    is_match = False
    for inner_list in correct_answer:
        for element in inner_list:
            #print(element)
            #print(str(correct_answer))
            if element in correct_answer or str(correct_answer) in element:
                is_match = True
                print(element)
                print(str(correct_answer))
                break
        if is_match:
            break
    if not is_match:
        false_positives += 1
true_negatives = 10 - true_positives - false_positives - false_negatives

precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)

print("true positive ",true_positives)
print("false positive ",false_positives)
print("false negative ",false_negatives)
print("true negative ",true_negatives)

print("precision = ", precision)
print("recall = ", recall)
print("F1 score = ", f1_score)

In [ ]:
from transformers import squad_convert_examples_to_features
from transformers import SquadV2Processor, squad_convert_examples_to_features
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Load SQuAD 2.0 dataset
processor = SquadV2Processor()
examples = load_dataset("squad_v2",split="validation")

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

# Tokenize and convert examples to features
features, dataset = squad_convert_examples_to_features(
    examples=examples,
    tokenizer=tokenizer,
    max_seq_length=384,
    doc_stride=128,
    max_query_length=64,
    is_training=False,
    return_dataset="pt",
    threads=1,
)

# Create DataLoader for the dataset
data_loader = DataLoader(dataset, batch_size=8)

# Function to check if a question is unanswerable
def is_unanswerable(scores, start_threshold=0.5, end_threshold=0.5):
    start_prob, start_index = torch.max(scores[0], dim=1)
    end_prob, end_index = torch.max(scores[1], dim=1)
    return start_prob.item() < start_threshold and end_prob.item() < end_threshold

# Extract unanswerable questions
unanswerable_questions = []

# Set model to evaluation mode
model.eval()

with torch.no_grad():
    for batch in tqdm(data_loader, desc="Extracting Unanswerable Questions"):
        inputs = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)

        # Get model outputs
        outputs = model(inputs, attention_mask=attention_mask)
        start_scores, end_scores = outputs.start_logits, outputs.end_logits

        # Check if the question is unanswerable
        for i in range(len(batch['example_id'])):
            if is_unanswerable((start_scores[i], end_scores[i])):
                unanswerable_questions.append(examples[batch['example_id'][i]].question_text)

# Print some extracted unanswerable questions
for i, question in enumerate(unanswerable_questions[:10]):
    print(f"{i+1}. {question}")
